In [1]:
import spacy 
from spacy.lang.en import English
from spacy.lang.es import Spanish
#from spacy.pipeline import EntityRuler
import json
import random
from spacy.tokens import Doc
from spacy.training import Example
from spacy.language import Language

In [19]:
#don't mess with the codeblock box
#1 NER Spacy create training data
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        #print(f)
        return(json.load(f))
    
    
def save_data (file, data):
    with open (file, "w", encoding='utf-8') as f:
       data = json.dump(data, f, indent=4)
       print(data)

def create_training_data(file, type):
    #print(type)
    #print(file)
    data = load_data(file)
    #print(data)
    patterns=[]
    for key, value in data.items():
        #print(key, value)
        for obj in value: #muni loop
            print(obj.get("Municipality"))
            pattern = {
                "label":type,
                "pattern": obj.get("Municipality")
                }
            patterns.append(pattern) # end of muni loop
        for obj in value: #primary county loop 
            if any(d.get("Municipality") == "Matamoros" for d in value): 
                print("found in municipality") 
            if not any(d.get("Municipality") == obj.get("Primary County/State") for d in value):
                print('not found in manicipality')
                pattern = {
                    "label":type,
                    "pattern": obj.get("Manicipality")
                    }
                patterns.append(pattern)
                if any(d.get("Primary County")=='Karnes' for d in value): 
                    print("found in Primary County/State")
        
            #print(pattern)
            #print(patterns)
    return(patterns)
patterns = create_training_data('../Corridos/data/TexasMunicipalities.json', 'GPE')
#print(patterns)

Houston
San Antonio
Dallas
Austin
Fort Worth
El Paso
Arlington
Corpus Christi
Plano
Lubbock
Laredo
Irving
Garland
Frisco
McKinney
Amarillo
Grand Prairie
Brownsville
Killeen
Pasadena
Denton
Mesquite
McAllen
Waco
Carrollton
Midland
Pearland
Abilene
Round Rock
College Station
Richardson
League City
Lewisville
Beaumont
Odessa
Sugar Land
Tyler
Allen
Wichita Falls
Edinburg
San Angelo
New Braunfels
Conroe
Bryan
Mission
Temple
Baytown
Longview
Pharr
Cedar Park
Flower Mound
Georgetown
Missouri City
Mansfield
Harlingen
North Richland Hills
San Marcos
Leander
Pflugerville
Victoria
Rowlett
Euless
Wylie
DeSoto
Port Arthur
Texas City
Galveston
Kyle
Burleson
Little Elm
Grapevine
Rockwall
Bedford
Cedar Hill
Huntsville
Haltom City
Keller
The Colony
Sherman
Waxahachie
Schertz
Coppell
Weslaco
Friendswood
Lancaster
Hurst
Duncanville
Rosenberg
Midlothian
Copperas Cove
Farmers Branch
Socorro
Texarkana
La Porte
San Juan
Del Rio
Prosper
Lufkin
Weatherford
Deer Park
Harker Heights
Cibolo
Nacogdoches
Cleburne
S

In [20]:
# start run here iff NEEDED. 
# don't mess with this codeblock box
#TX NER TRAINING SET 
#from 2 NER Spacy- Creating NER training data set Notebook

def generate_rules(patterns):
    nlp = English()
    #ruler = EntityRuler(nlp)
    ruler = nlp.add_pipe('entity_ruler', config={"validate": True})
    ruler.add_patterns(patterns) 
    nlp.to_disk("tx_trained_ner")

def test_model(model, text):
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        entities.append((ent.start_char, ent.end_char, ent.label_))
    if len(entities) >0: #if entity has been found move entity to results and entity list
        results = [text, {"entities": entities}]
        #print(results)
    return (results)

patterns = create_training_data('../Corridos/data/TexasMunicipalities.json', 'GPE')
generate_rules(patterns)
#print(patterns)

#how spacy wants to see read the data#
#TRAIN_DATA = [(text, {"entities": [(start, end, label)]})]#

nlp = spacy.load("tx_trained_ner")
TRAIN_DATA= []
with open ("../Corridos/corrido corpus/ElCorridodeGregorioCortez_X.txt", "r", encoding='utf-8')as f:
    text = f.read()
    #print(text)
    segments = text.split("\n\n")[0:]
    #cleaning up the lyric text. making it easier to read for the program
    for segment in segments:
        #segment = segment.strip() #might not need this code line 
        #segment = segement.replace("\n", " ")# might not need this code line
        #print(segment)
        results = test_model(nlp, segment)
        if results != []: #it found something and return it
            TRAIN_DATA.append(results)
        #print(results)

#print(TRAIN_DATA)
save_data("../Corridos/data/TxMunicipality_training_data.json", TRAIN_DATA)

Houston
San Antonio
Dallas
Austin
Fort Worth
El Paso
Arlington
Corpus Christi
Plano
Lubbock
Laredo
Irving
Garland
Frisco
McKinney
Amarillo
Grand Prairie
Brownsville
Killeen
Pasadena
Denton
Mesquite
McAllen
Waco
Carrollton
Midland
Pearland
Abilene
Round Rock
College Station
Richardson
League City
Lewisville
Beaumont
Odessa
Sugar Land
Tyler
Allen
Wichita Falls
Edinburg
San Angelo
New Braunfels
Conroe
Bryan
Mission
Temple
Baytown
Longview
Pharr
Cedar Park
Flower Mound
Georgetown
Missouri City
Mansfield
Harlingen
North Richland Hills
San Marcos
Leander
Pflugerville
Victoria
Rowlett
Euless
Wylie
DeSoto
Port Arthur
Texas City
Galveston
Kyle
Burleson
Little Elm
Grapevine
Rockwall
Bedford
Cedar Hill
Huntsville
Haltom City
Keller
The Colony
Sherman
Waxahachie
Schertz
Coppell
Weslaco
Friendswood
Lancaster
Hurst
Duncanville
Rosenberg
Midlothian
Copperas Cove
Farmers Branch
Socorro
Texarkana
La Porte
San Juan
Del Rio
Prosper
Lufkin
Weatherford
Deer Park
Harker Heights
Cibolo
Nacogdoches
Cleburne
S

In [21]:
def train_spacy(data, iterations): #passing training data/set and the number/generation of the training process. 30 is good. long time!!!!
    #print(data)
    TRAIN_DATA = data
    #print(TRAIN_DATA)
    nlp = spacy.blank("es")
    #print(nlp)
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner") #if there is no ner in the pipe, create a pipe
        nlp.add_pipe("ner", last=True)
    
    for _, annotations in TRAIN_DATA: #add labels
        print(annotations.get("entities"))
        for ent in annotations.get("entities"):
            ner.add_label(ent[2]) # i'm only working with one label 'GPE' if more increase i believe double check
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"] #get other names of other pipes to disable them during training. 
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iterations): # adjust the funtion not the items inside. arg of interations 30 is good
            print("Starting iteration "+ str(itn)) #where I am at in the interations process
            random.shuffle(TRAIN_DATA) #shuffle helps the program to learn not memorize order. 
            losses = {}
            for text, annotations in TRAIN_DATA:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations) #fix found here for ValueError: [E973] Unexpected type for NER data https://github.com/explosion/spaCy/issues/7038
                nlp.update([example], drop = 0.2, sgd= optimizer, losses=losses)            
            print(losses)  
    
    return(nlp) #model return

TRAIN_DATA = load_data("../Corridos/data/TxMunicipality_training_data.json")
print(TRAIN_DATA)
nlp = train_spacy(TRAIN_DATA, 30)
nlp.to_disk("tx_es_ner_model")

#time to test
test= "../Corridos/corrido corpus/la toma de Matamoros.txt"
with open(test, 'r', encoding='utf-8') as c:
    test = c.read()

import re

def clean_text(text):
    cleaned= re.sub(r"\n", " ", text)
    return(cleaned)
test = clean_text(test)
#print(test)

nlp = spacy.load("tx_ner_model")
doc =nlp(test) 
#print(doc)
for ent in doc.ents:
    print(ent.text, ent.label_)



[['He struck out for Gonzales\nWithout showing any fear,\n“Follow me, cowardly rangers, \nI am Gregorio Cortez.”', {'entities': [[18, 26, 'GPE']]}], ['Gregorio Cortez went out, \nHe went towards Laredo\nThey decided not to follow \nBecause they were afraid of him.', {'entities': [[43, 49, 'GPE']]}], ['Over by El Encinal, \nAccording to what we hear, \nThey made him a corral, \nAnd he killed them another sheriff. ', {'entities': [[11, 18, 'GPE']]}]]
[[18, 26, 'GPE']]
[[43, 49, 'GPE']]
[[11, 18, 'GPE']]
Starting iteration 0
{'ner': 61.584322929382324}
Starting iteration 1
{'ner': 51.23832294344902}
Starting iteration 2
{'ner': 31.024560764431953}
Starting iteration 3
{'ner': 11.09835914435098}
Starting iteration 4
{'ner': 5.431911043609944}
Starting iteration 5
{'ner': 5.842195047073538}
Starting iteration 6
{'ner': 4.667088787988618}
Starting iteration 7
{'ner': 2.5804394805152384}
Starting iteration 8
{'ner': 2.381309061329034}
Starting iteration 9
{'ner': 2.2740021840152167}
Starting 